In [57]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import os
import time
import sys
import torch.quantization
from pytorch_lightning import Trainer
import os
os.chdir("/home/k.schwienbacher/quantization-robustness")


%load_ext autoreload
%autoreload 2

from src.models.simple import SimpleModel
from src.models.lenet import LeNet
from src.models.resnet import ResNet18
from src.utils.model import load_experiment
from src.experiments.pruning import get_prunable_modules


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
def nonzero(tensor):
    """Returns absolute number of values different from 0
    Arguments:
        tensor {numpy.ndarray} -- Array to compute over
    Returns:
        int -- Number of nonzero elements
    """
    return np.sum(tensor != 0.0)


# https://pytorch.org/docs/stable/tensor_attributes.html
dtype2bits = {
    torch.float32: 32,
    torch.float: 32,
    torch.float64: 64,
    torch.double: 64,
    torch.float16: 16,
    torch.half: 16,
    torch.uint8: 8,
    torch.int8: 8,
    torch.int16: 16,
    torch.short: 16,
    torch.int32: 32,
    torch.int: 32,
    torch.int64: 64,
    torch.long: 64,
    torch.bool: 1,
}

def model_size(model, as_bits=False):
    """Returns absolute and nonzero model size
    Arguments:
        model {torch.nn.Module} -- Network to compute model size over
    Keyword Arguments:
        as_bits {bool} -- Whether to account for the size of dtype
    Returns:
        int -- Total number of weight & bias params
        int -- Out total_params exactly how many are nonzero
    """

    total_params = 0
    nonzero_params = 0
    for tensor in model.parameters():
        t = np.prod(tensor.shape)
        nz = nonzero(tensor.detach().cpu().numpy())
        if as_bits:
            bits = dtype2bits[tensor.dtype]
            t *= bits
            nz *= bits
        total_params += t
        nonzero_params += nz
    return int(total_params), int(nonzero_params)

In [59]:
#model = LeNet.load_from_checkpoint('/data/logs/kristian/runs/default/07-27-19/df1f105b-7584-4f48-b4d1-e7b03e27fefd/model_0.03125.ckpt')
model = ResNet18.load_from_checkpoint('/data/logs/kristian/runs/default/14-22-16/e11c17f0-1336-40cd-aedb-697485c51f89/model_0.03125.ckpt')


In [60]:
model05 = ResNet18.load_from_checkpoint('/data/logs/kristian/runs/default/14-22-16/e11c17f0-1336-40cd-aedb-697485c51f89/model_0.5.ckpt')


In [67]:
import copy
model = ResNet18()

for i in range(5):
    modules = get_prunable_modules(model.model)

    for module in modules:
        torch.nn.utils.prune.l1_unstructured(module, name="weight", amount=0.5)
    
   

    pruned_model = copy.deepcopy(model)
    modules = get_prunable_modules(pruned_model.model)
    for module in modules:
        torch.nn.utils.prune.remove(module, "weight")
    tot, nonz = model_size(model)
    print((tot-nonz) / tot)

0.4999995528384829
0.4999995528384829
0.4999995528384829
0.4999995528384829
0.4999995528384829


In [2]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

def evaluate(model, criterion, data_loader, neval_batches):
    model.eval()
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    cnt = 0
    with torch.no_grad():
        for image, target in data_loader:
            output = model(image)
            loss = criterion(output, target)
            cnt += 1
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            print('.', end = '')
            top1.update(acc1[0], image.size(0))
            top5.update(acc5[0], image.size(0))
            if cnt >= neval_batches:
                 return top1, top5

    return top1, top5

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [3]:
path = '/home/kredde/uni/quantization/logs/runs/2021-04-11/12-58-36'
num_eval_batches=1000
eval_batches_size=64

model, datamodule = load_experiment(path)
test_dataloader = datamodule.test_dataloader()
train_dataloader = datamodule.train_dataloader()

trainer = Trainer()
float_model = model
float_model

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


SimpleModel(
  (quant): QuantStub()
  (lin1): Linear(in_features=784, out_features=256, bias=True)
  (relu): ReLU()
  (lin2): Linear(in_features=256, out_features=10, bias=True)
  (dequant): DeQuantStub()
  (criterion): CrossEntropyLoss()
  (train_accuracy): Accuracy()
  (val_accuracy): Accuracy()
  (test_accuracy): Accuracy()
)

In [4]:
float_model.eval()
criterion = nn.CrossEntropyLoss()

In [11]:
print("Size of baseline model")
size = print_size_of_model(float_model)
top1, top5 = evaluate(float_model, criterion, test_dataloader, neval_batches=num_eval_batches)

Size of baseline model
Size (MB): 0.816019
.............................................................................................................................................................

In [12]:
print('Evaluation accuracy on %d images, %2.2f'%(num_eval_batches * eval_batches_size, top1.avg))

Evaluation accuracy on 64000 images, 6.98


In [13]:
trainer.test(float_model, test_dataloaders=[test_dataloader], verbose=False)

Testing: 100%|██████████| 157/157 [00:01<00:00, 112.95it/s]


[{'test/loss': 2.3150315284729004, 'test/acc': 0.0697999969124794}]

In [14]:
myModel, datamodule = load_experiment(path)
myModel.eval()

# Fuse Conv, bn and relu
myModel.fuse_model()


In [15]:
num_calibration_batches = 32

# Specify quantization configuration
# Start with simple min/max range estimation and per-tensor quantization of weights
myModel.qconfig = torch.quantization.default_qconfig
print(myModel.qconfig)
torch.quantization.prepare(myModel, inplace=True)

# Calibrate first
print('Post Training Quantization Prepare: Inserting Observers')
# print('\n Inverted Residual Block:After observer insertion \n\n', myModel.features[1].conv)

# Calibrate with the training set
evaluate(myModel, criterion, train_dataloader, neval_batches=num_calibration_batches)
print('Post Training Quantization: Calibration done')

# Convert to quantized model
torch.quantization.convert(myModel, inplace=True)
print('Post Training Quantization: Convert done')
# print('\n Inverted Residual Block: After fusion and quantization, note fused modules: \n\n',myModel.features[1].conv)

print("Size of model after quantization")
print_size_of_model(myModel)

top1, top5 = evaluate(myModel, criterion, test_dataloader, neval_batches=num_eval_batches)
print('Evaluation accuracy on %d images, %2.2f'%(num_eval_batches * eval_batches_size, top1.avg))

QConfig(activation=functools.partial(<class 'torch.quantization.observer.MinMaxObserver'>, reduce_range=True), weight=functools.partial(<class 'torch.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric))
Post Training Quantization Prepare: Inserting Observers
................................Post Training Quantization: Calibration done
Post Training Quantization: Convert done
Size of model after quantization
Size (MB): 0.208201
.............................................................................................................................................................Evaluation accuracy on 64000 images, 7.64


In [17]:
print_size_of_model(myModel)

Size (MB): 0.208201


In [18]:
test = trainer.test(myModel, test_dataloaders=[test_dataloader], verbose=False)
test

Testing: 100%|██████████| 157/157 [00:01<00:00, 112.61it/s]


[{'test/loss': 2.3441805839538574, 'test/acc': 0.07519999891519547}]